In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import json
from common_module import *
from plotting_module import *
import warnings

In [2]:
%matplotlib qt

set_matplotlib_defaults()

warnings.filterwarnings('ignore')

tf.keras.backend.set_floatx('float64')

In [3]:
# Load parameters
with open('../parameters.json', 'r') as f:
   parameters = json.load(f)['parameters']

parameters

{'R_lc': 16.67,
 'r_c': 13.336,
 'Pc_mode': 'free',
 'Pc_fixed': 0.075284943,
 'sep_width': 0.005,
 'coordinates': 'spherical',
 'n_1': 500,
 'n_2': 500,
 'extent': 3,
 'deriv_mode': 'automatic'}

In [4]:
# Grid
r, theta, q, mu, rho, z = get_grid(parameters)

if parameters['coordinates'] == 'spherical':

    x1, x2 = r, theta
    X, Z = r*np.sin(theta), r*np.cos(theta)

elif parameters['coordinates'] == 'compactified':

    x1, x2 = q, mu
    X, Z = np.sqrt(1 - mu **2) * q, mu * q

elif parameters['coordinates'] == 'cylindrical':

    x1, x2 = rho, z
    X, Z = rho, z

# PINN input
X_test = generate_test (x1, x2, parameters)

In [5]:
# Load trained network
tf.keras.utils.get_custom_objects()['AdaptiveActivation'] = AdaptiveActivation
N_p = tf.keras.models.load_model("../data/N_P.h5", compile=False)
N_t = tf.keras.models.load_model("../data/N_T.h5", compile=False)


# Calculate the scalar functions
P, Pc, T, T_prime = get_scalar_functions(X_test, N_p, N_t, parameters)

# Calculate the elements of the parametrisation
Np, Nt, fb, hb = get_pinn (X_test, P, Pc, N_p, N_t, parameters)

# Calculate magnetic and electric fields and related quantities
B_1, B_2, B_3, \
E_1, E_2, E_3, \
B_mag, E_mag, B_pol, \
E_dot_B, div_B, div_E = get_fields (x1, x2, P, T, parameters)

# Calculate the Pulsar Equation
pulsar_eq = pulsar_equation (x1, x2, P, T, T_prime, parameters)

In [6]:
# Generate analysis plots
plot_scalar_functions(X, Z, P, Pc, T, T_prime, parameters)
plot_pinn (X, Z, P, Pc, fb, hb, Np, parameters)
plot_fields (X, Z, P, Pc, B_mag, E_mag, div_E, parameters)
plot_pulsar_equation(X, Z, pulsar_eq, P, Pc, parameters)
plot_T_of_P (P, Pc, T, T_prime, parameters)

In [7]:
print(Pc)

0.10004350989337035


In [8]:
bla = np.linspace(1,10,5)
bla = tf.convert_to_tensor(bla)
with tf.GradientTape(persistent=True) as gt:
   gt.watch(bla)
   y = bla**2
   z = 2 * y

dy = gt.gradient(y, bla)
dz = gt.gradient(z, bla)
dzdy = gt.gradient (z, y)
print(y, dzdy)

tf.Tensor([  1.      10.5625  30.25    60.0625 100.    ], shape=(5,), dtype=float64) tf.Tensor([2. 2. 2. 2. 2.], shape=(5,), dtype=float64)
